# Handcrafted Features

## Imports

In [ ]:

########################### (https://github.com/curiousily/TensorFlow-on-Android-for-Human-Activity-Recognition-with-LSTMs/blob/master/human_activity_recognition.ipynb) imports
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42

#!pip install imbalanced-learn

from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from keras import backend as K
from sklearn.utils import shuffle

from matplotlib.pyplot import figure


########################### (https://github.com/curiousily/TensorFlow-on-Android-for-Human-Activity-Recognition-with-LSTMs/blob/master/human_activity_recognition.ipynb) imports
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle




# Upload Data 

---------------------------------



Train

In [ ]:
from google.colab import files
uploaded_signal_train = files.upload()

Test

In [ ]:
from google.colab import files
uploaded_signal_test = files.upload()

### Reading in segments & labels

Train

In [ ]:
# getting keys, which is file names of npy 
list_of_dataframes_train = [key for key in uploaded_signal_train.keys()]


# set up list to hold all loaded npy 
all_dataframe_train = [] 


for i in range(len(list_of_dataframes_train)):

    # load in the data 
    dataframe_train = pd.read_csv(list_of_dataframes_train[i])


    # append the data to 'all' list
    all_dataframe_train.append(dataframe_train) 


all_df_train = pd.concat(all_dataframe_train)

Test

In [ ]:
# getting keys, which is file names of npy 
list_of_dataframes_test = [key for key in uploaded_signal_test.keys()]


# set up list to hold all loaded npy 
all_dataframe_test = [] 


for i in range(len(list_of_dataframes_test)):

    # load in the data 
    dataframe_test = pd.read_csv(list_of_dataframes_test[i])


    # append the data to 'all' list
    all_dataframe_test.append(dataframe_test) 


all_df_test = pd.concat(all_dataframe_test)


# Quick Look 

In [ ]:
all_df_train.head()

In [ ]:
all_df_test.head()

# Train Test Split for each phase 

--------------------------------------------------


In [ ]:
all_df_train['Label_segment'].value_counts()

In [ ]:
all_df_test['Label_segment'].value_counts()

-------------------------------------

In [ ]:
dum_train = pd.get_dummies(all_df_train['Label_segment'] ,  prefix='Binary')
dum_test  = pd.get_dummies(all_df_test['Label_segment'] ,  prefix='Binary')

print(dum_train)

In [ ]:
# Concate the dummy variables back into dataframe 

all_df_train = pd.concat([all_df_train, dum_train] , axis=1)
all_df_test = pd.concat([all_df_test, dum_test] , axis=1)

all_df_train.head()

-------------------------

Quick Shuffle

In [ ]:
all_df_train = shuffle(all_df_train)
all_df_test = shuffle(all_df_test)

In [ ]:
all_df_train.columns

In [ ]:
# Getting X_train & y_train
X_train = all_df_train.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run', 'X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands' ,
                             'Binary_Go' , 'Binary_Walk 1', 'Binary_Turn 1', 'Binary_Walk 2', 'Binary_Turn 2', 'Binary_sit' 	], axis = 1)

In [ ]:
# Y Train Go
y_train_go = all_df_train['Binary_Go'].values

# Y Train Walk 1 
y_train_walk1 = all_df_train['Binary_Walk 1'].values

# Y Train Turn 1 
y_train_turn1 = all_df_train['Binary_Turn 1'].values

# Y Train Walk 2 
y_train_walk2 = all_df_train['Binary_Walk 2'].values

# Y Train Turn 2 
y_train_turn2 = all_df_train['Binary_Turn 2'].values

# Y Train Sit 
y_train_sit = all_df_train['Binary_sit'].values

In [ ]:
import imblearn
#print(imblearn.__version__)

from imblearn.under_sampling import RandomUnderSampler

rs = RandomUnderSampler(random_state=42 ,sampling_strategy ='majority')


In [ ]:
# fit and apply the transform
X_train_under_go, y_train_under_go = rs.fit_resample(X_train, y_train_go)


# fit and apply the transform
X_train_under_walk1, y_train_under_walk1 = X_train, y_train_walk1


# fit and apply the transform
X_train_under_walk2, y_train_under_walk2 = X_train, y_train_walk2


# fit and apply the transform
X_train_under_turn1, y_train_under_turn1 = X_train, y_train_turn1


# fit and apply the transform
X_train_under_turn2, y_train_under_turn2 = X_train, y_train_turn2

# fit and apply the transform
X_train_under_sit, y_train_under_sit = rs.fit_resample(X_train, y_train_sit)

--------------------------

Logic Check since using 2 pd.get_dums 

Go's will be coded as 0 and not Go 1 

In [ ]:
print(Counter(y_train_go))

print(Counter( y_train_under_go))

-----------------------------------

In [ ]:
# convert to dummy category
y_train_under_dum_go = pd.get_dummies(y_train_under_go)

# convert to dummy category
y_train_under_dum_walk1 = pd.get_dummies(y_train_under_walk1)


# convert to dummy category
y_train_under_dum_walk2 = pd.get_dummies(y_train_under_walk2)


# convert to dummy category
y_train_under_dum_turn1 = pd.get_dummies(y_train_under_turn1)

# convert to dummy category
y_train_under_dum_turn2 = pd.get_dummies(y_train_under_turn2)

# convert to dummy category
y_train_under_dum_sit = pd.get_dummies(y_train_under_sit)


# check one 
print(f'\nDummy labels Shape {y_train_under_dum_go.shape}')

------------------

In [ ]:
# Getting X_train & y_train

X_test =  all_df_test.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run',  'X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands' ,
                             'Binary_Go' , 'Binary_Walk 1', 'Binary_Turn 1', 'Binary_Walk 2', 'Binary_Turn 2', 'Binary_sit' 	], axis = 1)


In [ ]:
y_test_go =  all_df_test['Binary_Go'].values

y_test_walk1=  all_df_test['Binary_Walk 1'].values

y_test_walk2 =  all_df_test['Binary_Walk 2'].values

y_test_turn1 =  all_df_test['Binary_Turn 1'].values

y_test_turn2 =  all_df_test['Binary_Turn 2'].values

y_test_sit =  all_df_test['Binary_sit'].values


In [ ]:
# convert to dummy category
y_test_dum_go = pd.get_dummies(y_test_go)

# convert to dummy category
y_test_dum_walk1 = pd.get_dummies(y_test_walk1)

# convert to dummy category
y_test_dum_walk2 = pd.get_dummies(y_test_walk2)

# convert to dummy category
y_test_dum_turn1 = pd.get_dummies(y_test_turn1)

# convert to dummy category
y_test_dum_turn2 = pd.get_dummies(y_test_turn2)

# convert to dummy category
y_test_dum_sit = pd.get_dummies(y_test_sit)


# test one
print(f'\nDummy labels Shape {y_test_dum_go.shape}')

--------------------------------------

Standard Scale 

In [ ]:
# Set up ss for non-shuffled data
ss = StandardScaler()


X_train_under_go_scale = ss.fit_transform(X_train_under_go)

X_train_under_walk1_scale = ss.fit_transform(X_train_under_walk1)

X_train_under_walk2_scale = ss.fit_transform(X_train_under_walk2)

X_train_under_turn1_scale = ss.fit_transform(X_train_under_turn1)

X_train_under_turn2_scale = ss.fit_transform(X_train_under_turn2)

X_train_under_sit_scale = ss.fit_transform(X_train_under_sit)


X_test_scale = ss.transform(X_test)


----------------------

Shuffle Again

In [ ]:
from sklearn.utils import shuffle

X_train_under_go_scale    , y_train_under_dum_go    = shuffle(X_train_under_go_scale , y_train_under_dum_go , random_state= 0)

X_train_under_walk1_scale , y_train_under_dum_walk1 = shuffle(X_train_under_walk1_scale , y_train_under_dum_walk1 , random_state= 0)

X_train_under_walk2_scale, y_train_under_dum_walk2 = shuffle(X_train_under_walk2_scale, y_train_under_dum_walk2 , random_state= 0)

X_train_under_turn1_scale, y_train_under_dum_turn1 = shuffle(X_train_under_turn1_scale, y_train_under_dum_turn1 , random_state= 0)

X_train_under_turn2_scale, y_train_under_dum_turn2 = shuffle(X_train_under_turn2_scale, y_train_under_dum_turn2 , random_state= 0)

X_train_under_sit_scale,   y_train_under_dum_sit = shuffle(X_train_under_sit_scale, y_train_under_dum_sit , random_state= 0)



---------------------------

Deep 

In [ ]:
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from matplotlib import pyplot

from keras.optimizers import Adam

import time
import tensorflow as tf

------------------------------------------------------

# Go

In [ ]:
#n_timesteps, n_features, n_outputs = X_train.shape[1] , X_train.shape[2] , y_train_dum.shape[1]

n_outputs =  y_train_under_dum_go.shape[1]

# set up model
model = Sequential()

#input layer
model.add(Dense(100, activation='relu' , input_shape = (X_train_under_go_scale.shape[1], )))


# hidden layers
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

# output layer
model.add(Dense(n_outputs, activation='softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])

Early Stopping

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

 	
# patient early stopping
#es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=5 , min_delta=0.05)


earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.base_model_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')

Fit 

In [ ]:
# just for timing model
t0 = time.time()

# fitting model 
history = model.fit(X_train_under_go_scale, y_train_under_dum_go,
                    validation_data=(X_test_scale, y_test_dum_go ),
                    epochs = 100,
                    batch_size=200,
                    callbacks=[earlyStopping, mcp_save]
                    )


t1 = time.time()

total = t1-t0

print(f'Time Taken: {total}')

Learning Curves

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
accuracy = history_dict['categorical_accuracy']
val_accuracy = history_dict['val_categorical_accuracy']
 
epochs = range(1, len(loss_values) + 1)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
#
# Plot the model accuracy vs Epochs
#
ax[0].plot(epochs, accuracy, 'r', label='Training accuracy')
ax[0].plot(epochs, val_accuracy, 'b', label='Validation accuracy')
ax[0].set_title('Training & Validation Accuracy', fontsize=16)
ax[0].set_xlabel('Epochs', fontsize=16)
ax[0].set_ylabel('Accuracy', fontsize=16)
ax[0].legend()
#
# Plot the loss vs Epochs
#
ax[1].plot(epochs, loss_values, 'r', label='Training loss')
ax[1].plot(epochs, val_loss_values, 'b', label='Validation loss')
ax[1].set_title('Training & Validation Loss', fontsize=16)
ax[1].set_xlabel('Epochs', fontsize=16)
ax[1].set_ylabel('Loss', fontsize=16)
ax[1].legend()

------------------------

Reports

Make Predicitions

In [ ]:
# For Confusion matrix & Classification Report 

y_preds = model.predict(X_test_scale)
max_predictions = np.argmax(y_preds, axis=1)

max_test = np.argmax(np.asarray(y_test_dum_go), axis=1)



##################


y_preds_train = model.predict(X_train_under_go_scale)

max_predictions_train = np.argmax(y_preds_train, axis=1)
max_train = np.argmax(np.asarray(y_train_under_dum_go), axis=1)



# Metrics 

Test

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score

print("Test Accuracy : \t\t" ,accuracy_score(max_test, max_predictions))
print("Test Balanced Accuracy : \t" , balanced_accuracy_score(max_test, max_predictions))
print("Test F1 Score : \t\t" , f1_score(max_test, max_predictions , average='weighted'))

Train

In [ ]:
print("Train Accuracy : \t\t" ,accuracy_score(max_train, max_predictions_train))
print("Train Balanced Accuracy : \t" , balanced_accuracy_score(max_train, max_predictions_train))
print("Train F1 Score : \t\t" , f1_score(max_train, max_predictions_train, average='weighted'))

Imports 

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# set up labels 
LABELS = [ 'Not Go' , 'Go' ]


# Test Set 

In [ ]:
print(classification_report(max_test, max_predictions , target_names=LABELS))


# Train Set 

In [ ]:
print(classification_report(max_train, max_predictions_train , target_names=LABELS))

--------------------

# Confusion Matrix 

In [ ]:
confusion_matrix = metrics.confusion_matrix(max_test, max_predictions  )

confusion_matrix_t = metrics.confusion_matrix(max_train, max_predictions_train )

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS,  annot=True, fmt="d");
plt.title("Test Data Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix_t, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Train Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

-------------------------------------

<br>

------------------------------------------------------

# Walk 1 

In [ ]:
#n_timesteps, n_features, n_outputs = X_train.shape[1] , X_train.shape[2] , y_train_dum.shape[1]

n_outputs =  y_train_under_dum_go.shape[1]

# set up model
model_walk1 = Sequential()

#input layer
model_walk1.add(Dense(100, activation='relu' , input_shape = (X_train_under_go_scale.shape[1], )))


# hidden layers
model_walk1.add(Dense(50, activation='relu'))
model_walk1.add(Dropout(0.5))
model_walk1.add(Dense(50, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

# output layer
model_walk1.add(Dense(n_outputs, activation='softmax'))

# compile model
model_walk1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])

Early Stopping

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

 	
# patient early stopping
#es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=5 , min_delta=0.05)


earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.base_model_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')

Fit 

In [ ]:
# just for timing model
t0 = time.time()

# fitting model 
history_walk1 = model_walk1.fit(X_train_under_walk1_scale, y_train_under_dum_walk1,
                    validation_data=(X_test_scale, y_test_dum_walk1 ),
                    epochs = 100,
                    batch_size=20,
                    callbacks=[earlyStopping, mcp_save]
                    )


t1 = time.time()

total = t1-t0

print(f'Time Taken: {total}')

Learning Curves

In [ ]:
history_dict = history_walk1.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
accuracy = history_dict['categorical_accuracy']
val_accuracy = history_dict['val_categorical_accuracy']
 
epochs = range(1, len(loss_values) + 1)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
#
# Plot the model accuracy vs Epochs
#
ax[0].plot(epochs, accuracy, 'r', label='Training accuracy')
ax[0].plot(epochs, val_accuracy, 'b', label='Validation accuracy')
ax[0].set_title('Training & Validation Accuracy', fontsize=16)
ax[0].set_xlabel('Epochs', fontsize=16)
ax[0].set_ylabel('Accuracy', fontsize=16)
ax[0].legend()
#
# Plot the loss vs Epochs
#
ax[1].plot(epochs, loss_values, 'r', label='Training loss')
ax[1].plot(epochs, val_loss_values, 'b', label='Validation loss')
ax[1].set_title('Training & Validation Loss', fontsize=16)
ax[1].set_xlabel('Epochs', fontsize=16)
ax[1].set_ylabel('Loss', fontsize=16)
ax[1].legend()

------------------------

Reports

Make Predicitions

In [ ]:
# For Confusion matrix & Classification Report 

y_preds = model_walk1.predict(X_test_scale)
max_predictions = np.argmax(y_preds, axis=1)

max_test = np.argmax(np.asarray(y_test_dum_walk1), axis=1)

##################


y_preds_train = model_walk1.predict(X_train_under_walk1_scale)

max_predictions_train = np.argmax(y_preds_train, axis=1)
max_train = np.argmax(np.asarray(y_train_under_dum_walk1), axis=1)


# Metrics 

Test

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score

print("Test Accuracy : \t\t" ,accuracy_score(max_test, max_predictions))
print("Test Balanced Accuracy : \t" , balanced_accuracy_score(max_test, max_predictions))
print("Test F1 Score : \t\t" , f1_score(max_test, max_predictions , average='weighted'))

Train

In [ ]:
print("Train Accuracy : \t\t" ,accuracy_score(max_train, max_predictions_train))
print("Train Balanced Accuracy : \t" , balanced_accuracy_score(max_train, max_predictions_train))
print("Train F1 Score : \t\t" , f1_score(max_train, max_predictions_train, average='weighted'))

Imports 

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# set up labels 
LABELS = ['Not Walk 1', 'Walk 1']


# Test Set 

In [ ]:
print(classification_report(max_test, max_predictions , target_names=LABELS))


# Train Set 

In [ ]:
print(classification_report(max_train, max_predictions_train , target_names=LABELS))

--------------------

# Confusion Matrix 

In [ ]:
confusion_matrix = metrics.confusion_matrix(max_test, max_predictions  )

confusion_matrix_t = metrics.confusion_matrix(max_train, max_predictions_train )

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS,  annot=True, fmt="d");
plt.title("Test Data Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix_t, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Train Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

-------------------------------------

<br>

------------------------------------------------------

# Walk 2

In [ ]:
#n_timesteps, n_features, n_outputs = X_train.shape[1] , X_train.shape[2] , y_train_dum.shape[1]

n_outputs =  y_train_under_dum_go.shape[1]

# set up model
model_walk2 = Sequential()

#input layer
model_walk2.add(Dense(100, activation='relu' , input_shape = (X_train_under_go_scale.shape[1], )))


# hidden layers
model_walk2.add(Dense(50, activation='relu'))
model_walk2.add(Dropout(0.5))
model_walk2.add(Dense(50, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

# output layer
model_walk2.add(Dense(n_outputs, activation='softmax'))

# compile model
model_walk2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])

Early Stopping

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

 	
# patient early stopping
#es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=5 , min_delta=0.05)


earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.base_model_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')

Fit 

In [ ]:
# just for timing model
t0 = time.time()

# fitting model 
history_walk2 = model_walk2.fit(X_train_under_walk2_scale, y_train_under_dum_walk2,
                    validation_data=(X_test_scale, y_test_dum_walk2 ),
                    epochs = 100,
                    batch_size=200,
                    callbacks=[earlyStopping, mcp_save]
                    )


t1 = time.time()

total = t1-t0

print(f'Time Taken: {total}')

Learning Curves

In [ ]:
history_dict = history_walk2.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
accuracy = history_dict['categorical_accuracy']
val_accuracy = history_dict['val_categorical_accuracy']
 
epochs = range(1, len(loss_values) + 1)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
#
# Plot the model accuracy vs Epochs
#
ax[0].plot(epochs, accuracy, 'r', label='Training accuracy')
ax[0].plot(epochs, val_accuracy, 'b', label='Validation accuracy')
ax[0].set_title('Training & Validation Accuracy', fontsize=16)
ax[0].set_xlabel('Epochs', fontsize=16)
ax[0].set_ylabel('Accuracy', fontsize=16)
ax[0].legend()
#
# Plot the loss vs Epochs
#
ax[1].plot(epochs, loss_values, 'r', label='Training loss')
ax[1].plot(epochs, val_loss_values, 'b', label='Validation loss')
ax[1].set_title('Training & Validation Loss', fontsize=16)
ax[1].set_xlabel('Epochs', fontsize=16)
ax[1].set_ylabel('Loss', fontsize=16)
ax[1].legend()

------------------------

Reports

Make Predicitions

In [ ]:
# For Confusion matrix & Classification Report 

y_preds = model_walk2.predict(X_test_scale)
max_predictions = np.argmax(y_preds, axis=1)

max_test = np.argmax(np.asarray(y_test_dum_walk2), axis=1)

##################


y_preds_train = model_walk2.predict(X_train_under_walk2_scale)

max_predictions_train = np.argmax(y_preds_train, axis=1)
max_train = np.argmax(np.asarray(y_train_under_dum_walk2), axis=1)


# Metrics 

Test

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score

print("Test Accuracy : \t\t" ,accuracy_score(max_test, max_predictions))
print("Test Balanced Accuracy : \t" , balanced_accuracy_score(max_test, max_predictions))
print("Test F1 Score : \t\t" , f1_score(max_test, max_predictions , average='weighted'))

Train

In [ ]:
print("Train Accuracy : \t\t" ,accuracy_score(max_train, max_predictions_train))
print("Train Balanced Accuracy : \t" , balanced_accuracy_score(max_train, max_predictions_train))
print("Train F1 Score : \t\t" , f1_score(max_train, max_predictions_train, average='weighted'))

Imports 

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# set up labels 
LABELS = ['Not Walk 2', 'Walk 2']


# Test Set 

In [ ]:
print(classification_report(max_test, max_predictions,  target_names=LABELS))


# Train Set 

In [ ]:
print(classification_report(max_train, max_predictions_train , target_names=LABELS))

--------------------

# Confusion Matrix 

In [ ]:
confusion_matrix = metrics.confusion_matrix(max_test, max_predictions  )

confusion_matrix_t = metrics.confusion_matrix(max_train, max_predictions_train )

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS,  annot=True, fmt="d");
plt.title("Test Data Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix_t, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Train Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

-------------------------------------

<br>

------------------------------------------------------

# Turn 1 

In [ ]:
#n_timesteps, n_features, n_outputs = X_train.shape[1] , X_train.shape[2] , y_train_dum.shape[1]

n_outputs =  y_train_under_dum_go.shape[1]

# set up model
model_turn1 = Sequential()

#input layer
model_turn1.add(Dense(100, activation='relu' , input_shape = (X_train_under_go_scale.shape[1], )))


# hidden layers
model_turn1.add(Dense(50, activation='relu'))
model_turn1.add(Dropout(0.5))
model_turn1.add(Dense(50, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

# output layer
model_turn1.add(Dense(n_outputs, activation='softmax'))

# compile model
model_turn1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])

Early Stopping

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

 	
# patient early stopping
#es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=5 , min_delta=0.05)


earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.base_model_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')

Fit 

In [ ]:
# just for timing model
t0 = time.time()

# fitting model 
history_turn1 = model_turn1.fit(X_train_under_turn1_scale, y_train_under_dum_turn1,
                    validation_data=(X_test_scale, y_test_dum_turn1 ),
                    epochs = 100,
                    batch_size=200,
                    callbacks=[earlyStopping, mcp_save]
                    )


t1 = time.time()

total = t1-t0

print(f'Time Taken: {total}')

Learning Curves

In [ ]:
history_dict = history_turn1.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
accuracy = history_dict['categorical_accuracy']
val_accuracy = history_dict['val_categorical_accuracy']
 
epochs = range(1, len(loss_values) + 1)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
#
# Plot the model accuracy vs Epochs
#
ax[0].plot(epochs, accuracy, 'r', label='Training accuracy')
ax[0].plot(epochs, val_accuracy, 'b', label='Validation accuracy')
ax[0].set_title('Training & Validation Accuracy', fontsize=16)
ax[0].set_xlabel('Epochs', fontsize=16)
ax[0].set_ylabel('Accuracy', fontsize=16)
ax[0].legend()
#
# Plot the loss vs Epochs
#
ax[1].plot(epochs, loss_values, 'r', label='Training loss')
ax[1].plot(epochs, val_loss_values, 'b', label='Validation loss')
ax[1].set_title('Training & Validation Loss', fontsize=16)
ax[1].set_xlabel('Epochs', fontsize=16)
ax[1].set_ylabel('Loss', fontsize=16)
ax[1].legend()

------------------------

Reports

Make Predicitions

In [ ]:
# For Confusion matrix & Classification Report 

y_preds = model_turn1.predict(X_test_scale)
max_predictions = np.argmax(y_preds, axis=1)

max_test = np.argmax(np.asarray(y_test_dum_turn1), axis=1)

##################


y_preds_train = model_turn1.predict(X_train_under_turn1_scale)

max_predictions_train = np.argmax(y_preds_train, axis=1)
max_train = np.argmax(np.asarray(y_train_under_dum_turn1), axis=1)


# Metrics 

Test

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score

print("Test Accuracy : \t\t" ,accuracy_score(max_test, max_predictions))
print("Test Balanced Accuracy : \t" , balanced_accuracy_score(max_test, max_predictions))
print("Test F1 Score : \t\t" , f1_score(max_test, max_predictions , average='weighted'))

Train

In [ ]:
print("Train Accuracy : \t\t" ,accuracy_score(max_train, max_predictions_train))
print("Train Balanced Accuracy : \t" , balanced_accuracy_score(max_train, max_predictions_train))
print("Train F1 Score : \t\t" , f1_score(max_train, max_predictions_train, average='weighted'))

Imports 

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# set up labels 
LABELS = ['Not Turn 1', 'Turn 1']


# Test Set 

In [ ]:
print(classification_report(max_test, max_predictions , target_names=LABELS))


# Train Set 

In [ ]:
print(classification_report(max_train, max_predictions_train , target_names=LABELS))

--------------------

# Confusion Matrix 

In [ ]:
confusion_matrix = metrics.confusion_matrix(max_test, max_predictions  )

confusion_matrix_t = metrics.confusion_matrix(max_train, max_predictions_train )

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS,  annot=True, fmt="d");
plt.title("Test Data Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix_t, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Train Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

-------------------------------------

<br>

<br>

------------------------------------------------------

# Turn 2 

In [ ]:
#n_timesteps, n_features, n_outputs = X_train.shape[1] , X_train.shape[2] , y_train_dum.shape[1]

n_outputs =  y_train_under_dum_go.shape[1]

# set up model
model_turn2 = Sequential()

#input layer
model_turn2.add(Dense(100, activation='relu' , input_shape = (X_train_under_go_scale.shape[1], )))


# hidden layers
model_turn2.add(Dense(50, activation='relu'))
model_turn2.add(Dropout(0.5))
model_turn2.add(Dense(50, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

# output layer
model_turn2.add(Dense(n_outputs, activation='softmax'))

# compile model
model_turn2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])

Early Stopping

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

 	
# patient early stopping
#es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=5 , min_delta=0.05)


earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.base_model_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')

Fit 

In [ ]:
# just for timing model
t0 = time.time()

# fitting model 
history_turn2 = model_turn2.fit(X_train_under_turn2_scale, y_train_under_dum_turn2,
                    validation_data=(X_test_scale, y_test_dum_turn2 ),
                    epochs = 100,
                    batch_size=200,
                    callbacks=[earlyStopping, mcp_save]
                    )


t1 = time.time()

total = t1-t0

print(f'Time Taken: {total}')

Learning Curves

In [ ]:
history_dict = history_turn2.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
accuracy = history_dict['categorical_accuracy']
val_accuracy = history_dict['val_categorical_accuracy']
 
epochs = range(1, len(loss_values) + 1)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
#
# Plot the model accuracy vs Epochs
#
ax[0].plot(epochs, accuracy, 'r', label='Training accuracy')
ax[0].plot(epochs, val_accuracy, 'b', label='Validation accuracy')
ax[0].set_title('Training & Validation Accuracy', fontsize=16)
ax[0].set_xlabel('Epochs', fontsize=16)
ax[0].set_ylabel('Accuracy', fontsize=16)
ax[0].legend()
#
# Plot the loss vs Epochs
#
ax[1].plot(epochs, loss_values, 'r', label='Training loss')
ax[1].plot(epochs, val_loss_values, 'b', label='Validation loss')
ax[1].set_title('Training & Validation Loss', fontsize=16)
ax[1].set_xlabel('Epochs', fontsize=16)
ax[1].set_ylabel('Loss', fontsize=16)
ax[1].legend()

------------------------

Reports

Make Predicitions

In [ ]:
# For Confusion matrix & Classification Report 

y_preds = model_turn2.predict(X_test_scale)
max_predictions = np.argmax(y_preds, axis=1)

max_test = np.argmax(np.asarray(y_test_dum_turn2), axis=1)

##################


y_preds_train = model_turn2.predict(X_train_under_turn2_scale)

max_predictions_train = np.argmax(y_preds_train, axis=1)
max_train = np.argmax(np.asarray(y_train_under_dum_turn2), axis=1)


# Metrics 

Test

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score

print("Test Accuracy : \t\t" ,accuracy_score(max_test, max_predictions))
print("Test Balanced Accuracy : \t" , balanced_accuracy_score(max_test, max_predictions))
print("Test F1 Score : \t\t" , f1_score(max_test, max_predictions , average='weighted'))

Train

In [ ]:
print("Train Accuracy : \t\t" ,accuracy_score(max_train, max_predictions_train))
print("Train Balanced Accuracy : \t" , balanced_accuracy_score(max_train, max_predictions_train))
print("Train F1 Score : \t\t" , f1_score(max_train, max_predictions_train, average='weighted'))

Imports 

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# set up labels 
LABELS = ['Not Turn 2', 'Turn 2']


# Test Set 

In [ ]:
print(classification_report(max_test, max_predictions , target_names=LABELS))


# Train Set 

In [ ]:
print(classification_report(max_train, max_predictions_train , target_names=LABELS))

--------------------

# Confusion Matrix 

In [ ]:
confusion_matrix = metrics.confusion_matrix(max_test, max_predictions  )

confusion_matrix_t = metrics.confusion_matrix(max_train, max_predictions_train )

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS,  annot=True, fmt="d");
plt.title("Test Data Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix_t, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Train Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

-------------------------------------

<br>

------------------------------------------------------

# Sit

In [ ]:
#n_timesteps, n_features, n_outputs = X_train.shape[1] , X_train.shape[2] , y_train_dum.shape[1]

n_outputs =  y_train_under_dum_go.shape[1]

# set up model
model_sit = Sequential()

#input layer
model_sit.add(Dense(100, activation='relu' , input_shape = (X_train_under_go_scale.shape[1], )))


# hidden layers
model_sit.add(Dense(50, activation='relu'))
model_sit.add(Dropout(0.5))
model_sit.add(Dense(50, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

# output layer
model_sit.add(Dense(n_outputs, activation='softmax'))

# compile model
model_sit.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])

Early Stopping

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

 	
# patient early stopping
#es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=5 , min_delta=0.05)


earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.base_model_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')

Fit 

In [ ]:
# just for timing model
t0 = time.time()

# fitting model 
history_sit = model_sit.fit(X_train_under_sit_scale, y_train_under_dum_sit,
                    validation_data=(X_test_scale, y_test_dum_sit),
                    epochs = 100,
                    batch_size=200,
                    callbacks=[earlyStopping, mcp_save]
                    )


t1 = time.time()

total = t1-t0

print(f'Time Taken: {total}')

Learning Curves

In [ ]:
history_dict = history_sit.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
accuracy = history_dict['categorical_accuracy']
val_accuracy = history_dict['val_categorical_accuracy']
 
epochs = range(1, len(loss_values) + 1)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
#
# Plot the model accuracy vs Epochs
#
ax[0].plot(epochs, accuracy, 'r', label='Training accuracy')
ax[0].plot(epochs, val_accuracy, 'b', label='Validation accuracy')
ax[0].set_title('Training & Validation Accuracy', fontsize=16)
ax[0].set_xlabel('Epochs', fontsize=16)
ax[0].set_ylabel('Accuracy', fontsize=16)
ax[0].legend()
#
# Plot the loss vs Epochs
#
ax[1].plot(epochs, loss_values, 'r', label='Training loss')
ax[1].plot(epochs, val_loss_values, 'b', label='Validation loss')
ax[1].set_title('Training & Validation Loss', fontsize=16)
ax[1].set_xlabel('Epochs', fontsize=16)
ax[1].set_ylabel('Loss', fontsize=16)
ax[1].legend()

------------------------

Reports

Make Predicitions

In [ ]:
# For Confusion matrix & Classification Report 

y_preds = model_sit.predict(X_test_scale)
max_predictions = np.argmax(y_preds, axis=1)

max_test = np.argmax(np.asarray(y_test_dum_sit), axis=1)

##################


y_preds_train = model_sit.predict(X_train_under_sit_scale)

max_predictions_train = np.argmax(y_preds_train, axis=1)
max_train = np.argmax(np.asarray(y_train_under_dum_sit), axis=1)


# Metrics 

Test

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score

print("Test Accuracy : \t\t" ,accuracy_score(max_test, max_predictions))
print("Test Balanced Accuracy : \t" , balanced_accuracy_score(max_test, max_predictions))
print("Test F1 Score : \t\t" , f1_score(max_test, max_predictions , average='weighted'))

Train

In [ ]:
print("Train Accuracy : \t\t" ,accuracy_score(max_train, max_predictions_train))
print("Train Balanced Accuracy : \t" , balanced_accuracy_score(max_train, max_predictions_train))
print("Train F1 Score : \t\t" , f1_score(max_train, max_predictions_train, average='weighted'))

Imports 

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# set up labels 
LABELS = ['Not Sit', 'Sit']


# Test Set 

In [ ]:
print(classification_report(max_test, max_predictions , target_names=LABELS))


# Train Set 

In [ ]:
print(classification_report(max_train, max_predictions_train , target_names=LABELS))

--------------------

# Confusion Matrix 

In [ ]:
confusion_matrix = metrics.confusion_matrix(max_test, max_predictions  )

confusion_matrix_t = metrics.confusion_matrix(max_train, max_predictions_train )

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS,  annot=True, fmt="d");
plt.title("Test Data Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix_t, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Train Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

-------------------------------------

In [ ]:
# upload one dataframe to test logic of model 

# from google.colab import files
# uploaded_dataframe_test = files.upload()

In [ ]:
test_df = pd.read_csv('29_9_Participant_SlideSize_10_Handcrafted_Features_DF_Resampled_RCP')

In [ ]:
# Getting X_train & y_train
X_data = test_df.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run','X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands' ], axis = 1)
#X_data = signal_test.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run' ], axis = 1)

y_data = test_df['Label_segment'].values

# scale with already fit ss 
X_data_scale = ss.transform(X_data)

# convert to dummy category
y_data_dum = pd.get_dummies(y_data)

# Take max of dummy classifier
y_data = np.argmax(np.asarray(y_data_dum), axis=1)

In [ ]:
# Go 
go_model_preds = model.predict(X_data_scale)

# Walk 1
walk1_model_preds = model_walk1.predict(X_data_scale)

# Turn 1 
turn1_model_preds = model_turn1.predict(X_data_scale)

# Walk 2 
walk2_model_preds = model_walk2.predict(X_data_scale)

# Turn 2 
turn2_model_preds = model_turn2.predict(X_data_scale)

# Sit
sit_model_preds = model_sit.predict(X_data_scale)

------------------------------------------------

<br>

<br>